# 演習：SVMの実装

この演習課題では、python を使用してSVMをスクラッチから実装します。

In [134]:
import numpy as np
import matplotlib.pyplot as plt
import unittest

## SVM モデルの実装

まず、SVMにのコスト関数の実装を行います。コスト関数は次の式で表されます。

$$
J(\mathrm{w})=\frac{1}{2}\|\mathrm{w}\|+\frac{C}{N} \sum_{i}^{n} \max \left(0,1-y_{i} \left(\mathrm{w} \cdot x_{i}+b\right)\right)
$$

ここで、$w$ は重みパラメータ、$b$ はバイアス、$x_i$は入力データ、$y_i$は教師データを表します。$C$ はマージンの強さを指定するパラメータです。




### 演習1：コスト関数の実装（2点）
上記のコスト関数 $J(\mathrm{w})$ を実装してください。

関数の入力は
   - $w$：パラメータベクトル（特徴量の次元 x 1）
   - $X$：入力データ（サンプル数 x 特徴量の次元）
   - $Y$：教師データ（サンプル数 x 1）
   - $C$：マージンの強さ（スカラー）

とし、出力は
   - $J$：コスト（スカラー値）

とします。
   
   下に実装結果があっているか確認するコードを付けているので、これを活用しながら作成してみてください。

In [135]:
# ヒント：ベクトルのノルムは `np.linalg.norm` を使って計算できます。

def compute_cost(W, X, Y, C):
    # ここにコードを書く
    N = Y.shape[0]
    J = 1/2 * np.linalg.norm(W)
    for i in range(N):
        J += C/N * max(0, 1-Y[i]*(W@X[i].T))
    return J

In [136]:
# テスト用コード。OKと出れば正しく実装できている。
# 改変しないでください

class CheckCost(unittest.TestCase):
    def test_cost(self):
        np.random.seed(0)
        w = np.random.randn(10)
        X = np.random.randn(100, 10)
        Y = np.random.randn(100)
        cost = compute_cost(w, X, Y, 10)

        self.assertAlmostEqual(cost, 20.529, places=3)

if __name__ == '__main__': 
    unittest.main(argv=['first-arg-is-ignored', 'CheckCost.test_cost'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 演習2：勾配の計算

SVMのコスト関数の勾配を実装しましょう。勾配の式は次で与えられます

$$
\nabla_{w} J(w) = \frac{1}{N} \sum_{i}^{n} 
\begin{cases}w & \text { if } \max \left(0,1-y_{i} \left(w \cdot x_{i}\right)\right)=0 \\
\mathrm{w}-C y_{i} x_{i} & \text { otherwise }\end{cases}
$$

関数の入力は
   - $w$：パラメータベクトル（特徴量の次元 x 1）
   - $X$：入力データ（サンプル数 x 特徴量の次元）
   - $Y$：教師データ（サンプル数 x 1）
   - $C$：マージンの強さ（スカラー）

とし、出力は
   - $\nabla_{w} J(w)$：パラメータの勾配（特徴量の次元 x 1）

とします。

テスト用コードを使って正しく実装できているか確認してください。

In [137]:
def calculate_cost_gradient(W, X, Y, C):
    # ここにコードを書く
    dwJ = np.zeros(W.shape)
    N = Y.shape[0]
    for i in range(N):
        if max(0, 1-Y[i]*(W@X[i].T)) == 0:
            dwJ += W
        else:
            dwJ += W - C*Y[i]*X[i]
    dwJ /= N
    return dwJ

In [138]:
# テスト用コード。OKと出れば正しく実装できている。
# 改変しないでください

class CheckGradient(unittest.TestCase):
    def test_grad(self):
        np.random.seed(0)
        w = np.random.randn(3)
        X = np.random.randn(100, 3)
        Y = np.random.randn(100)

        grad = calculate_cost_gradient(w, X, Y, 10)

        assert np.allclose(grad, np.array([4.395, 0.241, 2.353]), rtol=1e-2)

if __name__ == '__main__': 
    unittest.main(argv=['first-arg-is-ignored', 'CheckGradient.test_grad'], exit=False)

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


### 演習3：勾配降下法の実装

これまでで作成した関数を使って、訓練データから SVMもパラメータ $w$ を学習します。この関数では次のことを行います。
   1. 与えられた訓練データからパラメータの勾配を計算する（`calculate_cost_gradient`）
   2. パラメータの勾配を使ってパラメータを更新する
   3. 一定の頻度でコストを計算し、コストが減少していっていることを確認する。
   - 1～3を繰り返す。





関数の入力は
   - `W`：パラメータベクトルの初期値（特徴量の次元 x 1）
   - `X`：入力データ（サンプル数 x 特徴量の次元）
   - `Y`：教師データ（サンプル数 x 1）
   - `C`：マージンの強さ（スカラー）
   - `max_epochs`：パラメータの更新回数
   - `learning_rate`：学習率

とし、出力は
   - `W`：学習されたパラメータベクトル（特徴量の次元 x 1）

としてください。

In [139]:
def sgd(W, X, Y, C, max_epochs, learning_rate):
    # ここにコードを書く

    return W

## 実データへの適用

ここまでで作成したSVMモデルを実データで学習・評価します。



### データの読み込み

ここでは、Kaggleで公開されているタイタニック号の乗船データを使用します。これには乗客の性別・年齢・宿泊した部屋のグレードなどの情報と、その乗客が生き延びたか否かのデータが入っています。

乗客が生き延びたか否かをほかの属性から予測できるか試してみましょう。


In [140]:
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

titanic = fetch_openml(data_id=40945, as_frame=True)['frame']
titanic

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,3.0,0,"Zabour, Miss. Hileni",female,14.5000,1.0,0.0,2665,14.4542,None,C,None,328.0,None
1305,3.0,0,"Zabour, Miss. Thamine",female,NaN,1.0,0.0,2665,14.4542,None,C,None,NaN,None
1306,3.0,0,"Zakarian, Mr. Mapriededer",male,26.5000,0.0,0.0,2656,7.2250,None,C,None,304.0,None
1307,3.0,0,"Zakarian, Mr. Ortin",male,27.0000,0.0,0.0,2670,7.2250,None,C,None,NaN,None


データについての詳しい説明は https://atmarkit.itmedia.co.jp/ait/articles/2007/02/news016.html を見てください

ここでは、入力に使用する属性として pclass, sex, age, sibsp, parch, fareを使用します。下のセルをそのまま実行してください。

In [141]:
# データの前処理

titanic['sex'] = titanic['sex'].map({'female': 0, 'male': 1})
titanic = titanic.iloc[:, [0, 1, 3, 4, 5, 6, 8]]
titanic = titanic.dropna(how='any')

X = titanic.iloc[:, [0, 2, 3, 4, 5, 6]].values
y = titanic['survived'].cat.codes.values.astype(np.float64)
y = y * 2 - 1

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

### 演習4：モデルの学習

作成したSVMをタイタニックの訓練データを使用して学習してください。

学習に使用するパラメータは、`C=10000, max_epochs=10000, learning_rate=1e-6` 程度がおすすめです。コストは単調には減少せず、多少変動しながら下がっていくことに注意してください。

### 演習5：モデルの評価

テストデータを使ってモデルの分類精度を評価してください。正しく実装できていれば75%程度になるはずです。

# 注意

- 前回と同様に、**提出前にメニューの「ランタイム」>「再起動してすべてのセルを実行」をクリックして、正しく動くことを確認してください。**